In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
from bs4 import BeautifulSoup
import html5lib
import requests
import datetime
import time

In [2]:
path = 'http://ergast.com/api/f1'
season = '2016'
race = '3'
URL = path + '/' + season + '/' + race + '/' + 'laps?limit=1000'

#Download
# results = requests.get(URL)
# soup = BeautifulSoup(results.content, 'lxml')
# print("First 1000 records downloaded")
# #Part 2
# URL = path + '/' + season + '/' + race + '/' + 'laps?limit=1000&offset=1000'
# results2 = requests.get(URL)
# soup2 = BeautifulSoup(results.content, 'lxml')

# with open('china_race_dl.xml', 'wb') as f:
#     f.write(results.content)
#     f.write(results2.content)


#open saved copy
with open('china_race_dl.xml', 'rb') as f:
    results = f.read()
soup = BeautifulSoup(results, 'lxml')

In [3]:
#pull in driverids to names
driver_info = pd.DataFrame.from_csv('driver_info.csv')
driverid_dict = {driver_info.ix[i]['driverid']:driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName'] for i in driver_info.index}
driverid_dict['sainz'] = driverid_dict['sainz'] + ' Jr.'
colors = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']: driver_info.ix[i]['color'] for i in driver_info.index}
colors['Carlos Sainz Jr.'] = colors['Carlos Sainz']

In [4]:
driver_abrev = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']:driver_info.ix[i]['code'] for i in driver_info.index}
driver_abrev['Carlos Sainz Jr.'] = 'SAI'

In [5]:
headers = ('driverid', 'lap', 'time')
num_rows = len(soup.find_all('timing'))
race_dict = {}
for header in headers:
    race_dict[header] = [race[header] for race in soup.find_all('timing')]
df = pd.DataFrame(race_dict)
df['minutes'] = [x.split(":")[0] for x in df['time']]
df['seconds'] = [x.split(":")[1] for x in df['time']]
df['lap_sec'] = df['minutes'].astype('int64') * 60 + df['seconds'].astype('float32')
df['lap'] = df['lap'].astype('int64')
df['driver'] = [driverid_dict[driver] for driver in df['driverid']]

In [6]:
pivot = df.pivot(index='driver', columns='lap', values='lap_sec')

In [7]:
for i in range(2, df['lap'].max() + 1):
    pivot.ix[:,i] = pivot.ix[:,i] + pivot.ix[:, i-1]

In [8]:
unpivot = pivot.stack()
new_df = unpivot.reset_index()
new_df.columns = ['driverid', 'lap', 'cume_sec']
new_df.sort_values('cume_sec', ascending=True, inplace=True)
new_df = new_df.reset_index()
new_df.drop('index', axis=1, inplace=True)

In [9]:
from f1 import Team
from f1 import Race

In [10]:
#Set up a Race object, score it, return a dataframe of all points
ThisRace = Race('China')
df_score = ThisRace.score()

C:\users\jason\documents\fantasy-formula-one\f1.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Team_Pts'][fantasy_points['Finish'].isin(team_points)] = 1
C:\users\jason\documents\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\users\jason\documents\fantasy-formula-one\f1.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy

In [11]:
#Set up current dataframe
current = pd.DataFrame(columns=['position', 'lap'])
current['position'] = df_score['Grid'].copy()
current['lap'] = 0
if 'DNS' in current['position']:
    current['position'][current['position'] == 'DNS'] = np.NaN
if 'PL' in current['position']:
    current['position'][current['position'] == 'PL'] = np.NaN
current.sort_values('position', inplace=True)

In [12]:
#Set all laps completed to zero
ThisRace.laps_completed = {driver:current.ix[driver]['lap'] for driver in current.index}

In [13]:
import matplotlib.animation as animation
fastest_lap_time = 1000
first = False

def update_driver_scores2(step, rects, ax):
    global fastest_lap_time
    global ThisRace
    global current
    global new_df
    global first
    if first == False:
        first = True
    else:
        driver, lap, cume_sec = new_df.ix[step]
        if lap == 1:
            if cume_sec != np.NaN:
                cur_lap_time = cume_sec
        elif new_df.ix[i, 'cume_sec'] != np.NaN:
            cur_lap_time = cume_sec - float(new_df['cume_sec'][(new_df['driverid'] == driver) & (new_df['lap'] == lap - 1)])

        if cur_lap_time < fastest_lap_time: #check for fastest lap
            ThisRace.fastest_lap = driver
            fastest_lap_time = cur_lap_time
        current.ix[driver, 'lap'] += 1 #increment lap
        current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable sort
        current['position'] = np.arange(1,len(current) + 1) #recalc positions
        ThisRace.laps_completed[driver] = lap #update laps completed for driver
        ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
        ThisRace.score().sort_values('Finish', inplace=True, ascending=True)   
        for rect, h, driver in zip(rects, 
                                   [ThisRace.fantasy_points[driver] for driver in current.index], 
                                   [driver for driver in current.index]):
            rect.set_height(h)
            rect.set_color(colors[driver])
        ax.xaxis.set_ticklabels([driver_abrev[driver] for driver in current.index])
        #ax.xaxis.set_ticklabels(current.index)
start_time = datetime.datetime.now()

#whole race
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()

fig1 = plt.figure()
#fig, ax = plt.subplots()
ax = fig1.add_subplot(1,1,1)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = ax.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, current.index, rotation=90)
plt.xlim([0, len(points)+1])
plt.title('Drivers Fantasy Points')
plt.ylabel('Points')
plt.ylim([0, 25])
ani = animation.FuncAnimation(fig1, update_driver_scores2, 50, fargs=([rects, ax]))
ani.save('China.mp4', fps=1)

C:\users\jason\documents\fantasy-formula-one\f1.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Team_Pts'][fantasy_points['Finish'].isin(team_points)] = 1
C:\users\jason\documents\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\users\jason\documents\fantasy-formula-one\f1.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy

In [14]:
fig, axarr = plt.subplots(2,2)
axarr[0,0].set_title('Driver Points')
axarr[0,0].plot(current.index, current['position'])
axarr[0,1].set_title('Team Points')
axarr[0,1].plot()
axarr[1,0].set_title('Team Points Over Time')

plt.show()

C:\users\jason\documents\fantasy-formula-one\f1.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Team_Pts'][fantasy_points['Finish'].isin(team_points)] = 1
C:\users\jason\documents\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\users\jason\documents\fantasy-formula-one\f1.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy

In [18]:
ThisTeam = 

AttributeError: 'Race' object has no attribute 'teams'

In [ ]:
import matplotlib.animation as animation

def update_line(num, data, line):
    line.set_data(data[...,:num])
    return line,

fig1 = plt.figure()

data = np.random.rand(2, 25)
l, = plt.plot([], [], 'r-')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('x')
plt.title('test')
line_ani = animation.FuncAnimation(fig1, update_line, 25, fargs=(data, l),
    interval=50, blit=True)
#line_ani.save('lines.mp4')

fig2 = plt.figure()

x = np.arange(-9, 10)
y = np.arange(-9, 10).reshape(-1, 1)
base = np.hypot(x, y)
ims = []
for add in np.arange(15):
    ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
    blit=True)
#im_ani.save('im.mp4', metadata={'artist':'Guido'})

plt.show()

In [ ]:
import matplotlib.animation as animation
def update_driver_scores(raceObj, current, driver, lap, lap_time, abs_time):
    #do I need globals so the objects are edited?
    current.ix[driver, 'lap'] += 1 #increment lap
    current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable
    current['position'] = np.arange(1,len(current) + 1) #recalc positions
    raceObj.laps_completed[driver] = lap #update laps completed for driver
    raceObj.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
    if lap_time < fastest_lap_time: #check for fastest lap
        raceObj.fastest_lap = driver
        fastest_lap_time = cur_lap_time
    return raceObj.score().sort_values('Finish', inplace=True, ascending=True)

def update_bar_chart(racePanda, rects):
    for rect, h in zip(rects, racePanda['Total_Race_Pts']):
        rect.set_height(h)

fastest_lap_time = 1000
start_time = datetime.datetime.now()
#whole race
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
i = 0
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()

fig1 = plt.figure()
ax = fig1.add_subplot(1,1,1)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = plt.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, current.index, rotation=90)
plt.xlim([-1, len(points)])
plt.title('Drivers Fantasy Points')
plt.xlabel('Driver')
plt.ylabel('Points')
plt.ylim([0, 25])
#ani = animation.FuncAnimation(fig1, update_driver_scores, i, fargs=(ThisRace, ))
plt.show()